<a href="https://colab.research.google.com/github/GreggRodgers02/Colab_notebooks/blob/main/ElevatorPitch_Resume_Point_Builder_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elevator Pitch: Resume Bullet Point Builder

## Setup

### Install Packages

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-core
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-co

### Import Libraries

In [ ]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.agents import create_agent
import os
from getpass import getpass
import json
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser

### Initiate AI Key

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key Here:")

Enter OpenAI API Key Here:··········


### Initiate AI Model

In [ ]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.3
    )

In [ ]:
agent = create_agent(model)

## Input Json

In [ ]:
user_input = {
    "experience": "As a senior project lead on a large-scale digital transformation initiative focused on redesigning a\
     company’s customer-facing web and mobile platforms to streamline onboarding and improve self-service, I oversaw a team\
      of 12, including 7 interns across product, marketing, and data analytics. I defined project scope, established\
       performance KPIs, and mentored interns through structured workflows and weekly performance reviews. Within four\
        months, the team delivered a fully integrated platform that increased user onboarding completion by 38%, reduced\
         customer support tickets by 26%, and grew weekly active users by 41%. Intern-led initiatives accounted for 30% of\
          implemented features, highlighting effective leadership, delegation, and talent development while maintaining\
           delivery velocity and quality benchmarks."
}

In [ ]:
user_input["experience"].lower()

'as a senior project lead on a large-scale digital transformation initiative focused on redesigning a     company’s customer-facing web and mobile platforms to streamline onboarding and improve self-service, i oversaw a team      of 12, including 7 interns across product, marketing, and data analytics. i defined project scope, established       performance kpis, and mentored interns through structured workflows and weekly performance reviews. within four        months, the team delivered a fully integrated platform that increased user onboarding completion by 38%, reduced         customer support tickets by 26%, and grew weekly active users by 41%. intern-led initiatives accounted for 30% of          implemented features, highlighting effective leadership, delegation, and talent development while maintaining           delivery velocity and quality benchmarks.'

## Pre-Processing

In [ ]:
pre_processing_json = json.dumps(user_input, indent=2)

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    '''
You are an information extraction and normalization engine for resume experience text.

Goal:
Convert the user’s single free-form prompt into a structured JSON object that downstream resume-bullet generators can use.

Hard rules:
- Use ONLY information explicitly present in the user prompt. Do not infer facts (e.g., tools, metrics, team size, seniority) unless directly stated.
- If a field is missing or unclear, set it to null or an empty list.
- Do not generate resume bullets. Only extract, normalize, and label facts.
- Never fabricate numbers. Only include metrics that appear in the user prompt.
- Preserve the user’s intent and wording where possible; paraphrase only to normalize.

Output requirements:
- Output MUST be valid JSON (no markdown, no commentary).
- Follow the schema exactly and include all top-level keys even if empty.
- Keep extracted strings concise (prefer short phrases over sentences).

Extraction tasks:
1) Detect experience level hint if present (entry|mid|senior). If not explicit, set null.
2) Extract STAR elements as best-effort:
   - Situation (context/background)
   - Task (responsibility/goal)
   - Actions (what the user did; verbs)
   - Results (outcomes/impact)
3) Extract metrics:
   - Any numbers, percentages, time saved, revenue, counts, growth, latency, etc.
   - Store the raw metric text and (if possible) a normalized numeric value + unit.
4) Extract entities:
   - Roles/titles, company/org, domain/industry, tools/tech, stakeholders, team size, scope
5) Identify ambiguity and missing details needed for strong bullets (especially quantifiable/leadership) and list follow-up questions.

JSON schema:
{{
  "experience_level": "entry" | "mid" | "senior" | null,
  "star": {{
    "situation": [string],
    "task": [string],
    "actions": [string],
    "results": [string]
  }},
  "metrics": {{
    "has_metrics": boolean,
    "items": [
      {{
        "raw": string,
        "value": "number" | null,
        "unit": string
      }}
    ]
  }}
}}
    '''
)

In [ ]:
#user_experience = user_input['experience']

In [ ]:
user_prompt = HumanMessagePromptTemplate.from_template(
    '''
Extract and normalize resume-experience facts from the following user prompt.

USER PROMPT (verbatim, do not edit):
{pre_processing_json}

    '''
)

In [ ]:
pre_processing_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [ ]:
pre_processing_chain = (
    pre_processing_prompt
    | model
)

In [ ]:
pre_processing_results = pre_processing_chain.invoke(
    {"pre_processing_json": pre_processing_json}
    )

In [ ]:
print(json.dumps(json.loads(pre_processing_results.content), indent=2))

{
  "experience_level": "senior",
  "star": {
    "situation": [
      "large-scale digital transformation initiative",
      "redesigning a company\u2019s customer-facing web and mobile platforms"
    ],
    "task": [
      "oversaw a team of 12, including 7 interns",
      "defined project scope",
      "established performance KPIs",
      "mentored interns"
    ],
    "actions": [
      "delivered a fully integrated platform"
    ],
    "results": [
      "increased user onboarding completion by 38%",
      "reduced customer support tickets by 26%",
      "grew weekly active users by 41%",
      "intern-led initiatives accounted for 30% of implemented features"
    ]
  },
  "metrics": {
    "has_metrics": true,
    "items": [
      {
        "raw": "38%",
        "value": 38,
        "unit": "%"
      },
      {
        "raw": "26%",
        "value": 26,
        "unit": "%"
      },
      {
        "raw": "41%",
        "value": 41,
        "unit": "%"
      },
      {
        "raw

## Bullet Point Generators

In [ ]:
pre_processing_results_json = json.loads(pre_processing_results.content)

In [ ]:
pre_processing_results_json

{'experience_level': 'senior',
 'star': {'situation': ['large-scale digital transformation initiative',
   'redesigning a company’s customer-facing web and mobile platforms'],
  'task': ['oversaw a team of 12, including 7 interns',
   'defined project scope',
   'established performance KPIs',
   'mentored interns'],
  'actions': ['delivered a fully integrated platform'],
  'results': ['increased user onboarding completion by 38%',
   'reduced customer support tickets by 26%',
   'grew weekly active users by 41%',
   'intern-led initiatives accounted for 30% of implemented features']},
 'metrics': {'has_metrics': True,
  'items': [{'raw': '38%', 'value': 38, 'unit': '%'},
   {'raw': '26%', 'value': 26, 'unit': '%'},
   {'raw': '41%', 'value': 41, 'unit': '%'},
   {'raw': '30%', 'value': 30, 'unit': '%'}]}}

### Action Verb Type

In [ ]:
action_verb_system_prompt = SystemMessagePromptTemplate.from_template(
    '''
You are a resume bullet generator specializing in the "Action Verb" style.

Goal:
Generate ONE high-quality resume bullet that starts with a strong action verb and is strictly grounded in the provided pre-processing JSON. The bullet point should be action orientated and focus on what has been done over quantifiable and leadership points

Hard rules (non-negotiable):
- Use ONLY facts present in the provided JSON. Do not invent tools, metrics, scope, team size, titles, or outcomes.
- If a needed detail is missing, do not fill it in. Write a bullet that remains truthful and specific to what is known.
- Do NOT include any numbers unless they appear in `metrics.items[].raw` (or are explicitly present in the JSON).
- Do NOT output multiple bullets.
- Do NOT output markdown or commentary.

Style requirements:
- Start with a strong past-tense action verb (e.g., Built, Led, Designed, Implemented, Optimized, Automated, Delivered).
- Prefer: Action + What + How + Why/Impact (only if supported by the JSON).
- Avoid fluff (e.g., “successfully”, “worked on”, “responsible for”).
- Use clear, resume-ready phrasing.

Output format:
Return valid JSON ONLY, exactly in this schema:
{{
  "type": "action_verb",
  "text": string,
  "warnings": [string]
}}

Warnings rules:
- If the JSON lacks any explicit result/impact, add a warning like: "No explicit results provided; bullet focuses on actions only."
- If STAR is sparse/ambiguous, add a warning like: "Limited detail provided."

    '''
)

In [ ]:
action_verb_user_prompt = HumanMessagePromptTemplate.from_template(
    '''
Generate one "Action Verb" resume bullet using ONLY the facts below.

PRE-PROCESSING JSON (source of truth):
{pre_processing_results_json}

    '''
)

In [ ]:
action_verb_prompt = ChatPromptTemplate.from_messages([action_verb_system_prompt, action_verb_user_prompt])

In [ ]:
action_verb_chain = (
    action_verb_prompt
    | model
)

In [ ]:
action_verb_results = action_verb_chain.invoke(
    {"pre_processing_results_json": pre_processing_results_json}
    )

In [ ]:
print(json.dumps(json.loads(action_verb_results.content), indent=2))

{
  "type": "action_verb",
  "text": "Delivered a fully integrated platform for a large-scale digital transformation initiative, overseeing a team of 12 and mentoring interns to enhance user experience.",
  "warnings": []
}


### Quantifiable Type

In [ ]:
quantifiable_system_prompt = SystemMessagePromptTemplate.from_template(
    '''
You are a resume bullet generator specializing in the "Quantifiable" style.

Goal:
Generate ONE resume bullet that emphasizes measurable impact, strictly grounded in the provided pre-processing JSON. The bullet point should focus on quantifiable facts  as the main impact point.


Hard rules (non-negotiable):
- Use ONLY facts present in the provided JSON. Do not invent tools, metrics, scope, team size, titles, timelines, or outcomes.
- Do NOT create or guess numbers. Only include numbers/percentages/counts if they appear in `metrics.items[].raw` OR are explicitly present elsewhere in the JSON.
- If no metrics exist, do NOT fabricate them. Instead:
  - Write "No metrics included in experience" AND
  - Add a warning indicating that metrics were not provided.
- Do NOT output multiple bullets.
- Do NOT output markdown or commentary.

Style requirements:
- Start with a strong past-tense action verb.
- Include a metric when available (%, $, time, count, latency, growth, etc.) and place it early when possible.
- Prefer: Action + What + Metric + How + Outcome (only if supported).
- Avoid vague claims (“significantly”, “greatly”) unless a metric supports it.

Output format:
Return valid JSON ONLY, exactly in this schema:
{{
  "type": "quantifiable",
  "text": string,
  "warnings": [string]
}}

Warnings rules:
- If `metrics.has_metrics` is false or metrics list is empty, include:
  "No measurable metrics found in the input; avoided inventing numbers."
- If results are missing/unclear, include:
  "No explicit results provided; bullet focuses on measurable-adjacent impact without claims."
    '''
)

In [ ]:
quantifiable_user_prompt = HumanMessagePromptTemplate.from_template(
    '''
Generate one "Quantifiable" resume bullet using ONLY the facts below.

PRE-PROCESSING JSON (source of truth):
{pre_processing_results_json}

    '''
)

In [ ]:
quantifiable_prompt = ChatPromptTemplate.from_messages([quantifiable_system_prompt, quantifiable_user_prompt])

In [ ]:
quantifiable_chain = (
    quantifiable_prompt
    | model
)

In [ ]:
quantifiable_results = quantifiable_chain.invoke(
    {"pre_processing_results_json": pre_processing_results_json}
    )

In [ ]:
print(json.dumps(json.loads(quantifiable_results.content), indent=2))

{
  "type": "quantifiable",
  "text": "Delivered a fully integrated platform that increased user onboarding completion by 38%, reduced customer support tickets by 26%, and grew weekly active users by 41%, while mentoring a team of 12, including 7 interns, who contributed to 30% of implemented features.",
  "warnings": []
}


### Leadership Type

In [ ]:
leadership_system_prompt = SystemMessagePromptTemplate.from_template(
    '''
You are a resume bullet generator specializing in the "Leadership" style.

Goal:
Generate ONE resume bullet that highlights leadership, ownership, or influence, strictly grounded in the provided pre-processing JSON.

Hard rules (non-negotiable):
- Use ONLY facts present in the provided JSON. Do not invent leadership, authority, team size, mentorship, decision-making, or ownership.
- Do NOT assume leadership based on seniority, verbs, or role titles unless explicitly stated in the JSON.
- If no explicit leadership signals are present, do NOT fabricate them.
  - Instead, write a bullet that emphasizes ownership, initiative, or cross-functional collaboration (only if supported).
  - If none of those are supported, return a conservative action-based bullet and include a warning.
- Do NOT output multiple bullets.
- Do NOT output markdown or commentary.

Leadership signals (must be explicit in the JSON):
- Led, owned, managed, mentored, coached
- Coordinated or influenced multiple people or teams
- Made decisions, set direction, or drove alignment
- Was accountable for delivery or outcomes

Style requirements:
- Start with a strong past-tense action verb (e.g., Led, Drove, Owned, Coordinated, Mentored).
- Prefer: Leadership Action + Who/What + How + Outcome (only if supported).
- Avoid vague claims (“provided leadership”, “key contributor”).

Output format:
Return valid JSON ONLY, exactly in this schema:
{{
  "type": "leadership",
  "text": string,
  "warnings": [string]
}}

Warnings rules:
- If no explicit leadership signals are found, include:
  "No explicit leadership indicators found in the input; bullet emphasizes ownership or collaboration only."
- If leadership scope is unclear (e.g., who was led, scale), include:
  "Leadership scope unclear; consider adding team size or responsibility."

    '''
)

In [ ]:
leadership_user_prompt = HumanMessagePromptTemplate.from_template(
    '''
Generate one "Leadership" resume bullet using ONLY the facts below.

PRE-PROCESSING JSON (source of truth):
{pre_processing_json}

    '''
)

In [ ]:
leadership_prompt = ChatPromptTemplate.from_messages([leadership_system_prompt, leadership_user_prompt])

In [ ]:
leadership_chain = (
    leadership_prompt
    | model
)

In [ ]:
leadership_results = leadership_chain.invoke(
    {"pre_processing_json": pre_processing_results_json}
    )

In [ ]:
print(json.dumps(json.loads(leadership_results.content), indent=2))

{
  "type": "leadership",
  "text": "Oversaw a team of 12, including 7 interns, during a large-scale digital transformation initiative, defining project scope and establishing performance KPIs, which resulted in a 38% increase in user onboarding completion and a 26% reduction in customer support tickets.",
  "warnings": []
}


## Inser Bullets within a JSON

In [ ]:
generated_bullets_dict = {}

In [ ]:
generated_bullets_dict["action_verb"] = json.loads(action_verb_results.content)

In [ ]:
generated_bullets_dict["quantifiable"] = json.loads(quantifiable_results.content)

In [ ]:
generated_bullets_dict["leadership"] = json.loads(leadership_results.content)

In [ ]:
generated_bullets_json = json.dumps(generated_bullets_dict, indent=2)

In [ ]:
print(generated_bullets_json)

{
  "action_verb": {
    "type": "action_verb",
    "text": "Delivered a fully integrated platform for a large-scale digital transformation initiative, overseeing a team of 12 and mentoring interns to enhance user experience.",
    "warnings": []
  },
  "quantifiable": {
    "type": "quantifiable",
    "text": "Delivered a fully integrated platform that increased user onboarding completion by 38%, reduced customer support tickets by 26%, and grew weekly active users by 41%, while mentoring a team of 12, including 7 interns, who contributed to 30% of implemented features.",
    "warnings": []
  },
  "leadership": {
    "type": "leadership",
    "text": "Oversaw a team of 12, including 7 interns, during a large-scale digital transformation initiative, defining project scope and establishing performance KPIs, which resulted in a 38% increase in user onboarding completion and a 26% reduction in customer support tickets.",
    "warnings": []
  }
}


## Judge Chain

In [ ]:
judge_system_prompt = SystemMessagePromptTemplate.from_template(
    '''
You are a resume bullet QA and grounding judge.

Goal:
Evaluate the quality of generated resume bullets and ensure they are strictly grounded in the provided pre-processing results (source of truth). Detect hallucinations and propose safe fixes.

Hard rules (non-negotiable):
- The pre_processing_results JSON is the ONLY allowed source of truth.
- A bullet is "hallucinated" if it contains ANY claim not explicitly supported by pre_processing_results (e.g., invented tools, metrics, scope, leadership, outcomes, timelines, stakeholders).
- Do NOT assume or infer facts beyond what is in pre_processing_results.
- Do NOT add new metrics or numbers unless they appear verbatim in pre_processing_results.metrics.items[].raw (or explicitly in other fields).
- Do NOT output markdown or commentary.

What to judge:
1) Grounding / factuality: every claim must be supported.
2) Type compliance:
   - action_verb: strong action verb, action-focused, no invented impact.
   - quantifiable: uses metrics only if present; if none, respond with "No metrics included in experience" and should include a warning.
   - leadership: leadership only if explicitly supported; otherwise must downgrade to ownership/collaboration and warn. Look for supported keywords from the source of truth such as lead, oversaw, leading and other key words.
3) Clarity + specificity: concise, concrete, avoids fluff.
4) Resume style: past tense and readable.

Leadership is considered explicitly supported if:
- A leadership verb (e.g., led, oversaw, managed, mentored) appears in pre_processing_results.star.actions[]
AND
- The leadership scope (team, interns, reports, contributors) appears in pre_processing_results.star.task[] or star.results[].

Leadership evidence may be distributed across multiple fields and does not need to appear as a single verbatim sentence.


Output requirements:
Return valid JSON ONLY in the following schema:

{{
  "passed": boolean,
  "summary": string,
  "bullets": [
    {{
      "type": "action_verb" | "quantifiable" | "leadership",
      "status": "pass" | "fail",
      "quality_score": integer,  // 1-5
      "issues": [
        {{
          "category": "hallucination" | "unsupported_metric" | "unsupported_tool" | "unsupported_leadership" | "vague" | "not_type_compliant" | "other",
          "claim": string,
          "reason": string,
          "evidence": [string]  // short snippets from pre_processing_results that support/contradict
        }}
      ],
      "patch": {{
        "action": "none" | "replace_text" | "edit_text",
        "replacement_text": string | null,
        "edit_instructions": [string]
      }},
      "warnings": [string]
    }}
  ]
}}

    '''
)

In [ ]:
judge_user_prompt = HumanMessagePromptTemplate.from_template(
    '''
Evaluate the generated resume bullets for grounding and quality.

SOURCE OF TRUTH (pre_processing_results):
{pre_processing_results}

GENERATED BULLETS (generated_bullets_json):
{generated_bullets_json}

Instructions:
- For each bullet, verify every claim is supported by pre_processing_results.
- Flag any hallucinations and type violations.
- Provide a patch for each bullet (replace_text or edit_text) that stays strictly within the source of truth.
- Return JSON only, following the schema in the system prompt.

    '''
)

In [ ]:
judge_prompt = ChatPromptTemplate.from_messages([judge_system_prompt, judge_user_prompt])

In [ ]:
judge_chain = (
    judge_prompt
    | model
)

In [ ]:
judge_results = judge_chain.invoke(
    {"pre_processing_results": pre_processing_results_json,
     "generated_bullets_json": generated_bullets_json}
    )

In [ ]:
print(pre_processing_results_json)

{'experience_level': 'senior', 'star': {'situation': ['large-scale digital transformation initiative', 'redesigning a company’s customer-facing web and mobile platforms'], 'task': ['oversaw a team of 12, including 7 interns', 'defined project scope', 'established performance KPIs', 'mentored interns'], 'actions': ['delivered a fully integrated platform'], 'results': ['increased user onboarding completion by 38%', 'reduced customer support tickets by 26%', 'grew weekly active users by 41%', 'intern-led initiatives accounted for 30% of implemented features']}, 'metrics': {'has_metrics': True, 'items': [{'raw': '38%', 'value': 38, 'unit': '%'}, {'raw': '26%', 'value': 26, 'unit': '%'}, {'raw': '41%', 'value': 41, 'unit': '%'}, {'raw': '30%', 'value': 30, 'unit': '%'}]}}


In [ ]:
print(json.dumps(json.loads(judge_results.content), indent=2))

{
  "passed": false,
  "summary": "All generated bullets contain hallucinations or unsupported claims.",
  "bullets": [
    {
      "type": "action_verb",
      "status": "fail",
      "quality_score": 3,
      "issues": [
        {
          "category": "unsupported_leadership",
          "claim": "overseeing a team of 12 and mentoring interns",
          "reason": "While the bullet mentions overseeing a team, it does not explicitly support the leadership scope of 'mentoring interns' as a leadership action.",
          "evidence": [
            "oversaw a team of 12, including 7 interns",
            "mentored interns"
          ]
        }
      ],
      "patch": {
        "action": "edit_text",
        "replacement_text": "Delivered a fully integrated platform for a large-scale digital transformation initiative, overseeing a team of 12 to enhance user experience.",
        "edit_instructions": [
          "Remove 'and mentoring interns' to avoid unsupported leadership claim."
      

## Finalizer Chain

In [ ]:
finalizer_system_prompt = SystemMessagePromptTemplate.from_template(
    '''
You are a deterministic resume bullet post-processor.

Goal:
Apply the judge_results to the generated_bullets_json to produce a final, corrected output called resume_point_builder_output.

Role clarity:
- You do NOT judge quality.
- You do NOT invent new content.
- You do NOT introduce new facts.
- You ONLY apply changes explicitly instructed by judge_results.

Hard rules (non-negotiable):
- generated_bullets_json is the baseline input.
- judge_results is the ONLY authority for modifications.
- Do NOT add, remove, or change bullets unless explicitly instructed in judge_results.patch.
- Do NOT invent new wording beyond what is provided in replacement_text or edit instructions.
- Preserve bullet ordering and bullet types.
- Preserve existing warnings unless the judge explicitly removes or replaces them.
- Do NOT output markdown or commentary.

Patch application rules:
- If patch.action == "none":
  - Leave the bullet unchanged.
- If patch.action == "replace_text":
  - Replace the bullet text entirely with patch.replacement_text.
- If patch.action == "edit_text":
  - Apply the edit instructions conservatively to the existing text.
  - If an edit is ambiguous or unsafe, prefer removing unsupported claims rather than adding new ones.
- If patch.replacement_text is null:
  - Keep the original bullet text unchanged and append a warning indicating insufficient information.

Output requirements:
Return valid JSON ONLY in the following schema:

{{
  "experience_level": string | null,
  "bullets": [
    {{
      "type": "action_verb" | "quantifiable" | "leadership",
      "text": string,
      "warnings": [string]
    }}
  ]
}}

Final output rules:
- The output must be safe, factual, and grounded.
- All hallucinated claims identified by the judge must be removed or neutralized.
- If judge_results.passed == true, the output should match generated_bullets_json exactly.

    '''
)

In [ ]:
finalizer_user_prompt = HumanMessagePromptTemplate.from_template(
    '''
Apply the judge results to the generated resume bullets.

GENERATED BULLETS (baseline input):
{generated_bullets_json}

JUDGE RESULTS (patch instructions):
{judge_results}

Instructions:
- Apply all patch instructions exactly as specified.
- Produce the final resume_point_builder_output JSON.
- Return JSON only.

    '''
)

In [ ]:
finalizer_prompt = ChatPromptTemplate.from_messages([finalizer_system_prompt, finalizer_user_prompt])

In [ ]:
finalizer_chain = (
    finalizer_prompt
    | model
)

In [ ]:
resume_point_builder_output = finalizer_chain.invoke(
    {"generated_bullets_json": generated_bullets_json,
     "judge_results": judge_results}
    )

## Final Output

In [ ]:
print(json.dumps(json.loads(resume_point_builder_output.content), indent=2))

{
  "experience_level": null,
  "bullets": [
    {
      "type": "action_verb",
      "text": "Delivered a fully integrated platform for a large-scale digital transformation initiative, overseeing a team of 12 to enhance user experience.",
      "warnings": []
    },
    {
      "type": "quantifiable",
      "text": "Delivered a fully integrated platform that increased user onboarding completion by 38%, reduced customer support tickets by 26%, and grew weekly active users by 41%.",
      "warnings": []
    },
    {
      "type": "leadership",
      "text": "Oversaw a team of 12, including 7 interns, during a large-scale digital transformation initiative.",
      "warnings": []
    }
  ]
}
